# TODO
Specify

FOLDS: number of folds to generate

DATA: string path to csv containing patient ID, survival status and tissue extraction site marker.

In [ ]:
FOLDS = 10
DATA = "tcga-brca-survival-for-cv.csv"

In [1]:
import preservedsite.crossfolds as cv
import pandas as pd

In [ ]:
data = pd.read_csv(DATA, dtype=str)

In [4]:
data = cv.generate(data, "Overall Survival Status", ["0:LIVING", "1:DECEASED"], crossfolds=FOLDS, patient_column='Patient ID', site_column='site', randomseed=42)

Crossfold 1: 0:LIVING - 85 1:DECEASED - 15  Sites: ['A2']
Crossfold 2: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AN', 'AO', 'PE']
Crossfold 3: 0:LIVING - 79 1:DECEASED - 5  Sites: ['EW', 'GM', 'OL']
Crossfold 4: 0:LIVING - 77 1:DECEASED - 11  Sites: ['4H', '5T', 'AR', 'LD', 'LQ', 'MS', 'OK', 'PL', 'W8', 'XX', 'Z7']
Crossfold 5: 0:LIVING - 78 1:DECEASED - 5  Sites: ['3C', '5L', 'AQ', 'C8', 'GI', 'JL', 'LL', 'WT']
Crossfold 6: 0:LIVING - 83 1:DECEASED - 7  Sites: ['A7', 'AC']
Crossfold 7: 0:LIVING - 97 1:DECEASED - 45  Sites: ['BH']
Crossfold 8: 0:LIVING - 78 1:DECEASED - 6  Sites: ['A8', 'S3']
Crossfold 9: 0:LIVING - 77 1:DECEASED - 5  Sites: ['D8', 'HN', 'UL', 'UU']
Crossfold 10: 0:LIVING - 83 1:DECEASED - 6  Sites: ['E2']


/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [5]:
data

,Patient ID,Sample ID,Number of Samples Per Patient,Diagnosis Age,Sex,Prior Cancer Diagnosis Occurence,Menopause Status,Race Category,ER Status By IHC,PR status by ihc,Fraction Genome Altered,American Joint Committee on Cancer Tumor Stage Code,Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Metastasis Stage Code,Overall Survival Status,Overall Survival (Months),Image Path,site,CV
0,TCGA-3C-AALI,TCGA-3C-AALI-01,1,50.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,BLACK OR AFRICAN AMERICAN,Positive,Positive,0.7164,2,1,0,0:LIVING,131.57,/shared/fsartori/MuSEPI/dataset/TCGA-3C-AALI-0...,3C,5
1,TCGA-3C-AALJ,TCGA-3C-AALJ-01,1,62.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,BLACK OR AFRICAN AMERICAN,Positive,Positive,0.534,2,1,0,0:LIVING,48.42,/shared/fsartori/MuSEPI/dataset/TCGA-3C-AALJ-0...,3C,5
2,TCGA-3C-AALK,TCGA-3C-AALK-01,1,52.0,Female,No,NaN,BLACK OR AFRICAN AMERICAN,Positive,Positive,0.0764,1,0,0,0:LIVING,47.57,/shared/fsartori/MuSEPI/dataset/TCGA-3C-AALK-0...,3C,5
3,TCGA-4H-AAAK,TCGA-4H-AAAK-01,1,50.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.2364,2,2,0,0:LIVING,11.43,/shared/fsartori/MuSEPI/dataset/TCGA-4H-AAAK-0...,4H,4
4,TCGA-5L-AAT0,TCGA-5L-AAT0-01,1,42.0,Female,Yes,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.0702,2,0,0,0:LIVING,48.52,/shared/fsartori/MuSEPI/dataset/TCGA-5L-AAT0-0...,5L,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,TCGA-WT-AB44,TCGA-WT-AB44-01,1,77.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.3436,1,0,X,0:LIVING,29.01,/shared/fsartori/MuSEPI/dataset/TCGA-WT-AB44-0...,WT,5
920,TCGA-XX-A899,TCGA-XX-A899-01,1,46.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.0625,1,2,X,0:LIVING,15.34,/shared/fsartori/MuSEPI/dataset/TCGA-XX-A899-0...,XX,4
921,TCGA-XX-A89A,TCGA-XX-A89A-01,1,68.0,Female,No,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.1764,3,0,X,0:LIVING,16.03,/shared/fsartori/MuSEPI/dataset/TCGA-XX-A89A-0...,XX,4
922,TCGA-Z7-A8R5,TCGA-Z7-A8R5-01,1,61.0,Female,Yes,Post (prior bilateral ovariectomy OR >12 mo si...,WHITE,Positive,Positive,0.0452,3,1,X,0:LIVING,107.98,/shared/fsartori/MuSEPI/dataset/TCGA-Z7-A8R5-0...,Z7,4


In [6]:
data["CV"] = data["CV"].astype(dtype=int)

In [7]:
import os

In [9]:
os.environ['OMP_NUM_THREADS'] = "10"

In [10]:
train_imgs = []
val_imgs = []
test_imgs = []
for i in range(1,FOLDS+1):
    test_idxs = list(data[data["CV"] == i]["Image Path"].apply(lambda x: str(x).replace("/shared/fsartori/MuSEPI/dataset/","").replace(".svs","")))
    test_imgs.append(test_idxs)
    without_test = data[data["CV"] != i].drop(columns=["CV"], axis=1)
    without_test = cv.generate(without_test, "Overall Survival Status", ["0:LIVING", "1:DECEASED"], crossfolds=9, patient_column='Patient ID', site_column='site', randomseed=42)
    without_test["CV"] = without_test["CV"].astype(dtype=int)
    val_idxs = list(without_test[without_test["CV"] == 9]["Image Path"].apply(lambda x: str(x).replace("/shared/fsartori/MuSEPI/dataset/","").replace(".svs","")))
    val_imgs.append(val_idxs)
    train_idxs = list(without_test[without_test["CV"] != 9]["Image Path"].apply(lambda x: str(x).replace("/shared/fsartori/MuSEPI/dataset/","").replace(".svs","")))
    train_imgs.append(train_idxs)
    


/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Crossfold 1: 0:LIVING - 77 1:DECEASED - 11  Sites: ['4H', '5T', 'AR', 'GI', 'LQ', 'MS', 'OK', 'S3', 'UL']
Crossfold 2: 0:LIVING - 79 1:DECEASED - 5  Sites: ['EW', 'GM', 'OL']
Crossfold 3: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AN', 'AO', 'PE']
Crossfold 4: 0:LIVING - 83 1:DECEASED - 7  Sites: ['A7', 'AC']
Crossfold 5: 0:LIVING - 83 1:DECEASED - 6  Sites: ['E2']
Crossfold 6: 0:LIVING - 78 1:DECEASED - 5  Sites: ['3C', 'AQ', 'C8', 'LD', 'LL', 'PL']
Crossfold 7: 0:LIVING - 97 1:DECEASED - 45  Sites: ['BH']
Crossfold 8: 0:LIVING - 78 1:DECEASED - 5  Sites: ['D8', 'HN', 'JL', 'UU']
Crossfold 9: 0:LIVING - 77 1:DECEASED - 6  Sites: ['5L', 'A8', 'W8', 'WT', 'XX', 'Z7']
Crossfold 1: 0:LIVING - 77 1:DECEASED - 5  Sites: ['D8', 'HN', 'LQ', 'UU']
Crossfold 2: 0:LIVING - 83 1:DECEASED - 6  Sites: ['E2']
Crossfold 3: 0:LIVING - 85 1:DECEASED - 15  Sites: ['A2']
Crossfold 4: 0:LIVING - 77 1:DECEASED - 11  Sites: ['3C', '5L', 'AR', 'JL', 'LD', 'MS', 'OK', 'UL', 'W8', 'Z7']
Crossfold 5: 0:LIVING - 97 

/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Crossfold 1: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AQ', 'C8', 'GI', 'JL', 'LD', 'LL', 'OK']
Crossfold 2: 0:LIVING - 78 1:DECEASED - 6  Sites: ['A8', 'S3']
Crossfold 3: 0:LIVING - 77 1:DECEASED - 11  Sites: ['3C', '5L', '5T', 'AR', 'LQ', 'MS', 'PL', 'W8', 'WT', 'XX', 'Z7']
Crossfold 4: 0:LIVING - 83 1:DECEASED - 7  Sites: ['A7', 'AC']
Crossfold 5: 0:LIVING - 85 1:DECEASED - 15  Sites: ['A2']
Crossfold 6: 0:LIVING - 78 1:DECEASED - 5  Sites: ['4H', 'AN', 'AO', 'PE']
Crossfold 7: 0:LIVING - 77 1:DECEASED - 5  Sites: ['D8', 'HN', 'UL', 'UU']
Crossfold 8: 0:LIVING - 97 1:DECEASED - 45  Sites: ['BH']
Crossfold 9: 0:LIVING - 83 1:DECEASED - 6  Sites: ['E2']


/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


...

/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Crossfold 1: 0:LIVING - 85 1:DECEASED - 15  Sites: ['A2']
Crossfold 2: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AN', 'AO', 'PE']
Crossfold 3: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AQ', 'C8', 'LL', 'S3']
Crossfold 4: 0:LIVING - 97 1:DECEASED - 45  Sites: ['BH']
Crossfold 5: 0:LIVING - 78 1:DECEASED - 11  Sites: ['4H', '5T', 'AR', 'JL', 'LD', 'LQ', 'MS', 'OK', 'W8', 'WT', 'XX', 'Z7']
Crossfold 6: 0:LIVING - 79 1:DECEASED - 5  Sites: ['EW', 'GM', 'OL']
Crossfold 7: 0:LIVING - 83 1:DECEASED - 6  Sites: ['E2']
Crossfold 8: 0:LIVING - 83 1:DECEASED - 7  Sites: ['A7', 'AC']
Crossfold 9: 0:LIVING - 78 1:DECEASED - 6  Sites: ['3C', '5L', 'A8', 'GI', 'PL']


/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Crossfold 1: 0:LIVING - 78 1:DECEASED - 6  Sites: ['A8', 'S3']
Crossfold 2: 0:LIVING - 83 1:DECEASED - 7  Sites: ['A7', 'AC']
Crossfold 3: 0:LIVING - 77 1:DECEASED - 5  Sites: ['D8', 'HN', 'MS', 'UU']
Crossfold 4: 0:LIVING - 97 1:DECEASED - 45  Sites: ['BH']
Crossfold 5: 0:LIVING - 78 1:DECEASED - 11  Sites: ['4H', '5L', '5T', 'AR', 'GI', 'JL', 'LQ', 'OK', 'PL', 'WT', 'XX', 'Z7']
Crossfold 6: 0:LIVING - 77 1:DECEASED - 5  Sites: ['AN', 'AO', 'PE']
Crossfold 7: 0:LIVING - 77 1:DECEASED - 5  Sites: ['3C', 'AQ', 'C8', 'LD', 'LL', 'UL', 'W8']
Crossfold 8: 0:LIVING - 79 1:DECEASED - 5  Sites: ['EW', 'GM', 'OL']
Crossfold 9: 0:LIVING - 85 1:DECEASED - 15  Sites: ['A2']


/shared/cgorner/environments/torch/lib/python3.10/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [14]:
len(train_imgs[2]) + len(val_imgs[2]) + len(test_imgs[2])

924

In [15]:
for i in range(0,FOLDS):
    df = pd.DataFrame()
    df["train"] = train_imgs[i]
    df["val"] = val_imgs[i] + [None]*(len(train_imgs[i]) - len(val_imgs[i]))
    df["test"] = test_imgs[i] + [None]*(len(train_imgs[i]) - len(test_imgs[i]))
    df.to_csv(f"splits_{i}.csv")